<a href="https://colab.research.google.com/github/Sauhardya007/Scrabble_analysis.ipynb/blob/main/Cat_v_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d tongpython/cat-and-dog




Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
cat-and-dog.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q cat-and-dog.zip -d cat_and_dog


replace cat_and_dog/test_set/test_set/cats/_DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,BatchNormalization
from keras import regularizers
from keras import Sequential
from tensorflow.keras.layers import (RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomTranslation, Rescaling)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/cat_and_dog/training_set/training_set',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)
test_ds=keras.utils.image_dataset_from_directory(
    directory='/content/cat_and_dog/test_set/test_set',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)
class_names = train_ds.class_names
print(class_names)


Found 8005 files belonging to 2 classes.
Found 2023 files belonging to 2 classes.
['cats', 'dogs']


In [ ]:
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(normalize_img)
test_ds=test_ds.map(normalize_img)

In [ ]:


model = Sequential()
#Data_augmentation_start
model.add(RandomFlip("horizontal"))
model.add(RandomRotation(0.1, fill_mode='nearest'))
model.add(RandomZoom(0.2, fill_mode='nearest'))
model.add(RandomTranslation(0.1, 0.1, fill_mode='nearest'))
model.add(RandomContrast(0.2))
#model.add(Rescaling(1./255))
#Stop

model.add(Conv2D(16, kernel_size=(3,3), strides=2,activation='relu', input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))  # reasonable

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.15))


model.add(Flatten())
model.add(Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
#model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))  # final layer should NOT have dropout


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:

history= model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6813 - loss: 0.6579 - val_accuracy: 0.6441 - val_loss: 0.7029
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6763 - loss: 0.6630 - val_accuracy: 0.7148 - val_loss: 0.7039
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.6925 - loss: 0.6962 - val_accuracy: 0.7444 - val_loss: 0.6141
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 16s 58ms/step - accuracy: 0.6929 - loss: 0.6628 - val_accuracy: 0.7593 - val_loss: 0.6050
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.6923 - loss: 0.6882 - val_accuracy: 0.7331 - val_loss: 0.6147
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.7157 - loss: 0.6298 - val_accuracy: 0.7622 - val_loss: 0.5934
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 20s 56ms/step - accuracy: 0.7080 - loss: 0.6326 - val_accuracy: 0.7088 - val_loss: 0.6302
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.7191 - loss: 0.6284 - 